# 유튜브 크롤링 하기


>  ## 1. 유튜브 API 호출하기


[유튜브크롤링 - 참고자료 - API호출하기](https://blog.naver.com/doublet7411/221511344483)


- (1단계) GCP (Google Cloud Platform) 접속
    - https://console.developers.google.com
    
- (2단계) API 라이브러리에서 "Youtube Data API v3" 선택
    - 사용 클릭
    - 좌측 열쇄모양 아이콘(사용자 인증정보) 클릭
    - 상단 `+사용자 인증정보 만들기` 버튼 클릭 -> `API 키` 클릭
    - API키 복사하기


- (3단계) 구글 포스트맨을 활용하여 정상적으로 호출되는지 확인
    - 구글포스트맨: 크롬 웹스토어에서 `Tabbed Postman` 설치
    - 크롬 확장프로그램 `Tabbed Postman`을 실행.
    - url:  `https://www.googleapis.com/youtube/v3/search`
    - 요청방식: `GET`
    - `URL params 버튼` 클릭
    - URL parameter: `key`
    - Value: `(2단계)에서 발급받은 API키`
 

- postman 사용 결과

![](./results/using_postman.JPG)

> ## 2. 조건에 맞는 유튜브 JSON형식 데이터 조회

- [유튜브 크롤링 참고자료 - 유튜브 크롤링2](https://blog.naver.com/doublet7411/221514043955)

- Tabbed postman

    - (URL Parameter Key) `key`  => (Value) `유튜브 api에서 발급받은 키`
    - (URL Parameter Key) `part`  => (Value) `snippet`
    - (URL Parameter Key) `order`  => (Value) `date`
    - (URL Parameter Key) `q`  => (Value) `찾고자하는 유튜브 이름 (쿼리)`
    
<br>
    
- 파이썬 패키지 설치
    - apiclient
    ```
    pip install apiclient
    ```

    - oauth2client
    ```
    pip install oauth2client
    ```

In [1]:
!pip install apiclient

  Stored in directory: C:\Users\USER\AppData\Local\pip\Cache\wheels\cc\7a\2d\1b55fbcfe08f95cec4bbef4c6626b79a11be7dc1a5124ddcf7
Successfully built apiclient


In [2]:
!pip install oauth2client

In [7]:
!pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in c:\users\user\anaconda3\lib\site-packages (1.9.3)


In [1]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser

In [2]:
DEVELOPER_KEY= "(2단계)에서 발급받은 유튜브 API"
YOUTUBE_API_SERVICE_NAME="youtube"
YOUTUBE_API_VERSION="v3"



def youtube_search(query):
    print('검색결과 => ', query)
    
    # 크롤링 객체
    youtube= build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, 
                   developerKey= DEVELOPER_KEY)
    
    # 검색결과 크롤링
    # 검색리스트를 부른다.
    # 쿼리에 명시된것과 일치한 결과물을 메소드에게 반환.
    search_response= youtube.search().list(
        q=query,
        part="snippet",
        maxResults=50
    ).execute()
    
    videos=[]
    channels=[]
    playlists=[]
    
    
    # 리스트에 적절한 각각의 결과물들을 추가한다.
    # 매칭된 비디오, 채널, 플레이리스트들을 리스트형태로 나타낸다.
    
    for search_result in search_response.get('items', []):
        if search_result['id']['kind']=='youtube#video':
            videos.append('%s (%s)' % (search_result['snippet']['title'],
                                       search_result['id']['videoId']))
            
        elif search_result['id']['kind']=='youtube#channel':
            channels.append('%s (%s)' %(search_result['snippet']['title'],
                                       search_result['id']['channelId']))
        elif search_result['id']['kind']=='youtube#playlist':
            playlists.append('%s (%s)' %(search_result['snippet']['title'],
                                        search_result['id']['playlistId']))
    return videos, channels, playlists

In [9]:

search_queries=['서울 강남 맛집', '서울 홍대 맛집', '서울 가성비 맛집',
               '서울 종로 맛집', '경기도 맛집', '서울 건국대 맛집']

videos, channels, playlists = youtube_search(search_queries[0])
videos

검색결과 =>  서울 강남 맛집


['[삼카슐랭] #10 서울 강남맛집 종합편 (with 쓰복만) (1tD2I9XO3-s)',
 '❤무한도전이 극찬한 강남의 맛있는집!!! 어육장 ㅣ강남역,역삼역 6분거리ㅣ강남맛집ㅣ하얀트리 (kP6uWsZedms)',
 '강남 맛집 여기다 있음 (W_NgtlZ1m0A)',
 '[한국오빠]믿고 가는 이영자 추천 서울 맛집 10곳 완벽정리 (배고픔주의) (Dav8BVt6nH8)',
 '[강남 맛집] 강남인들만 안다는 중국집을 다녀와봤습니다/중국냉면 맛집 /in gangnam style Chinese dish (ZpXaNIa57uA)',
 '강남 노포 추억의 맛집 베스트 3 #08 (HUKBbChbWII)',
 '011 향방 (서울 강남구 신사역) 중화요리 맛집 리뷰 &amp; 짬뽕 리뷰  a.k.a. 대통령들의 맛집 (CMlge7i-7kg)',
 '서울 최고의 부촌 강남구 완전 정복 (ft.서울여행,강남맛집)(korea seoul,travel seoul,travel korea) (3zhrGREtAA8)',
 '🚗서울강남편 🍚강남맛집추천 신논현역 \U0001f9aa🍤상무초밥 에서ㅣ일상브리로그 (N0xGxxdXCIQ)',
 '논현역 맛집 &#39;회식&#39;/횟집추천/강남맛집추천/논현동맛집/서울맛집/강남모임장소/리뷰퀸 (4RdvT1set7c)',
 '[돼동여지도]강남 맛집 (II_9DpUZOMQ)',
 '서울 강남 맛집 브이로그 | 돈까스, 수플레 팬케이크, 팟타이 | Yoonpic 윤픽 (QLoKRpWfit4)',
 '[하슐랭로드 4화] 강남가면 이거 꼭 드세요!! 백종원 핫도그에 방송나온 떡볶이를 맛보다! (yQq_0ABbnhE)',
 '강남 논현동 청평갈비 맛집! 강남맛집!! (C_bBbykEx-o)',
 '[맛집 소개] 강남역 맛집 TOP 10 (eEjn6wktrc4)',
 '강남 맛집 탐방! [마루가메 제면, 은행골, 오코노미야끼 우와] 솔직 리뷰 먹방 (8ISoaFBC9GM)',
 '강남에서 찾기힘든 한정식집 맛집발견! 70년 전통의 &#39;조양관&#39

In [4]:
youtube_search(search_queries[0])

In [ ]:


for query in search_queries:
    youtube_search(query)